In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
tender_data = pd.read_excel('./data/history1.xlsx')

In [3]:
# Menggabungkan atribut 'Kategori', 'Sumber Dana', dan 'Satker' menjadi satu kolom teks
tender_data['Content'] = tender_data['Kategori'] + ' ' + tender_data['Sumber Dana'] + ' ' + tender_data['Satker']

# Membuat mapping antara nama paket tender dengan indeks
tender_indices = dict(zip(tender_data['Nama Paket'], tender_data.index))

# Menambahkan kolom indeks pada dataset
tender_data['Index'] = tender_data['Nama Paket'].map(tender_indices)

# Membagi data menjadi training dan validation sets
train_data, val_data = train_test_split(tender_data[['Index', 'Content']], test_size=0.2, random_state=42)

In [4]:
# Jumlah unik paket tender
num_tenders = len(tender_indices)

# Membangun model rekomendasi menggunakan TensorFlow
input_layer = Input(shape=(num_tenders,))
hidden_layer = Dense(128, activation='relu')(input_layer)
output_layer = Dense(num_tenders, activation='softmax')(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)

In [5]:
# Menentukan optimizer dan compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



In [6]:
# Mengubah data training dan validation menjadi one-hot encoded vectors
def convert_to_one_hot(data):
    one_hot_data = tf.one_hot(data['Index'], depth=num_tenders)
    return one_hot_data

train_X = convert_to_one_hot(train_data)
val_X = convert_to_one_hot(val_data)

# Melabeli data target
train_y = train_X
val_y = val_X

In [7]:
# Melatih model
history = model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=15, batch_size=32)

Epoch 1/15
1488/1488 [==============================] - 266s 178ms/step - loss: 10.2867 - accuracy: 0.0021 - val_loss: 10.4958 - val_accuracy: 0.0066
Epoch 2/15
1488/1488 [==============================] - 255s 171ms/step - loss: 9.9162 - accuracy: 0.0120 - val_loss: 11.0784 - val_accuracy: 0.0161
Epoch 3/15
1488/1488 [==============================] - 251s 169ms/step - loss: 9.5213 - accuracy: 0.0250 - val_loss: 12.1227 - val_accuracy: 0.0267
Epoch 4/15
1488/1488 [==============================] - 252s 169ms/step - loss: 8.9999 - accuracy: 0.0414 - val_loss: 13.2521 - val_accuracy: 0.0370
Epoch 5/15
1488/1488 [==============================] - 250s 168ms/step - loss: 8.0994 - accuracy: 0.0812 - val_loss: 15.5761 - val_accuracy: 0.0570
Epoch 6/15
1488/1488 [==============================] - 250s 168ms/step - loss: 6.6182 - accuracy: 0.2927 - val_loss: 16.6984 - val_accuracy: 0.0819
Epoch 7/15
1488/1488 [==============================] - 250s 168ms/step - loss: 5.2679 - accuracy: 0.5345

In [19]:
# Fungsi untuk merekomendasikan paket tender berdasarkan paket yang telah diminati sebelumnya
def recommend_tenders(tender_title, model, tender_indices, top_n=10):
    # Mendapatkan indeks paket tender yang sesuai dengan judul yang diberikan
    tender_index = tender_indices[tender_title]

    # Mengubah input menjadi one-hot encoded vector
    input_data = tf.one_hot([tender_index], depth=num_tenders)

    # Menggunakan model untuk membuat prediksi
    predictions = model.predict(input_data)

    # Mendapatkan indeks N paket tender teratas yang direkomendasikan
    top_recommended_indices = tf.argsort(predictions, direction='DESCENDING')[0][:top_n]

    # Mengambil nama paket tender yang direkomendasikan
    recommended_tenders = tender_data['Nama Paket'].iloc[top_recommended_indices]

    return recommended_tenders

In [21]:
# Contoh penggunaan: merekomendasikan 5 paket tender teratas berdasarkan paket yang telah diminati sebelumnya
tender_title = 'Lomba Karya Jurnalistik'
recommended_tenders = recommend_tenders(tender_title, model, tender_indices, top_n=50)
filtered_tenders = recommended_tenders[tender_data['Status'] == 1]

# Menampilkan hasil rekomendasi
print("Hasil Rekomendasi untuk Paket Tender:", tender_title)
print(filtered_tenders)

1/1 [==============================] - 0s 34ms/step
Hasil Rekomendasi untuk Paket Tender: Lomba Karya Jurnalistik
52999    Pengadaan Kendaraan Roda-4 Bagian Kepegawaian ...
54294    Pengadaan Linen Jas Operasi Unit Pelayanan Lau...
53512    Pengadaan Konsultan Perencanaan Pembangunan Ru...
52958    Pengadaan Kendaraan PKP-PK Type IV termasuk Pe...
49757    Pengadaan Jasa Konstruksi Pembangunan Gedung P...
48567                      Pengadaan Hepafilter Ruang CSSD
52656     Pengadaan Kebutuhan Prisma TPE dll untuk TA 2018
51360          Pengadaan Jasa Layanan Foto Copy Tahun 2016
Name: Nama Paket, dtype: object


In [ ]:
# Display only the top 5 filtered tenders
top_5_filtered_tenders = filtered_tenders.head(5)
print(top_5_filtered_tenders)